# PREPARE DATA FOR ANALYSES: Get and format results data

This notebook explain how to 
It must be run **AFTER** :
- All previous notebooks (number 01 to 08) (extract of solutions)
- After retrieving all solutions: [https://doi.org/10.57745/OS1JND](https://doi.org/10.57745/OS1JND)

> Note:
>
> The Seed2lp supllement data are available[https://doi.org/10.57745/OS1JND](https://doi.org/10.57745/OS1JND)
>
> After downloadind and unzipping the package
> - analyses/results/metabolites_iCN718
> - analyses/results/supp_data
> - analyses/results/timer_one_sol

## Requirements
Module *seed2lp* needed

> Advice:
> 
> Use a conda env called s2lp

In [ ]:
!pip install seed2lp

# GET AND CONCAT SCOPES DATA

## **Slurm-based cluster**: Reproducing paper data
Slurm-based scripts for cluster are available:
- Launch if needed 
    - all previous scripts to get results, see notebook: 
        - [03_run_seed2lp](03_run_seed2lp.ipynb)
        - [05_run_netseed](05_run_netseed.ipynb)
        - [07_run_seed2lp_iCN718](07_run_seed2lp_iCN718.ipynb)
- Change **_source_** variable by the path of your conda environement with seed2lp installed in files: 
    - [10_1_job_run_scope_analyse_iCN718](../../scripts/plafrim_cluster/10_1_job_run_scope_analyse_iCN718.sh)
    - [10_1_job_run_scope_analyse_netseed](../../scripts/plafrim_cluster/10_1_job_run_scope_analyse_netseed.sh)
    - [10_1_job_run_scope_analyse_s2lp](../../scripts/plafrim_cluster/10_1_job_run_scope_analyse_s2lp.sh)
- launch 
    - [10_1_job_run_scope_analyse_iCN718](../../scripts/plafrim_cluster/10_1_job_run_scope_analyse_iCN718.sh): `sbatch 10_1_job_run_scope_analyse_iCN718.`
    - [10_1_job_run_scope_analyse_netseed](../../scripts/plafrim_cluster/10_1_job_run_scope_analyse_netseed.sh): `sbatch 10_1_job_run_scope_analyse_netseed.`
    - [10_1_job_run_scope_analyse_s2lp](../../scripts/plafrim_cluster/10_1_job_run_scope_analyse_s2lp.sh): `sbatch 10_1_job_run_scope_analyse_s2lp.`

## Function

In [20]:
from os import path, listdir, makedirs, walk
from shutil import copyfile

In [4]:
analyse_dir = "../../analyses"
data_dir  = f"{analyse_dir}/data/"
result_dir=f"{analyse_dir}/results"

objective_dir=f"{data_dir}/objective"
sbml_dir=f"{data_dir}/bigg/sbml"
e_coli_dir=f"{data_dir}/bigg/sbml_e_coli_core"

In [5]:
if not path.isdir(e_coli_dir):
    makedirs(e_coli_dir)
    copyfile(path.join(sbml_dir, "e_coli_core.xml"), path.join(e_coli_dir, "e_coli_core.xml"))
copyfile(path.join(sbml_dir, "iCN718.xml"), path.join(e_coli_dir, "iCN718.xml"))

'../../data/bigg/sbml_e_coli_core/iCN718.xml'

In [5]:
def run_scope_analyses(sbml_dir:str, scope_dir:str, obj_dir:str,
                       lev1:list, lev2:list, lev3:list, lev4:list, 
                       given_species:str=None):
    file = "../../scripts/10_1_scope_analyse.py"
    for filename in listdir(sbml_dir):
        species = f'{path.splitext(path.basename(filename))[0]}'
        if not given_species or species==given_species:
            sbml_path = path.join(sbml_dir, filename)
            objective_path = path.join(obj_dir,f"{species}_target.txt")
            dir = list(map(lambda x: x.split('.')[0], listdir(scope_dir)))
            if species in dir:
                path_seed=path.join(scope_dir, species, 'sbml')
                path_scope=path.join(scope_dir, species, 'scope')
                for l1 in lev1:
                    for l2 in lev2:
                        for l3 in lev3:
                            for l4 in lev4:
                                modes_info = path.join(l1, l2, l3, l4)
                                full_path_seed = path.join(path_seed, modes_info)
                                full_path_scope = path.join(path_scope, modes_info)
                                if path.isdir(full_path_scope):
                                    command=f"{species} {sbml_path} {full_path_scope} {full_path_seed} {objective_path} {modes_info}"
                                    !python {file} {command}

## Seed2LP

In [6]:
list_dir_lev1=["full", "target"]
list_dir_lev2=["reasoning", "reasoning_filter", "reasoning_guess_check", "reasoning_guess_check_diversity"]
list_dir_lev3=["minimize", "subset_minimal"]
list_dir_lev4=["accu", "no_accu"]

### Variable to change (if wanted)

In [15]:
scope_dir_s2lp=f"{result_dir}/scopes_s2lp"

### RUN

In [14]:
run_scope_analyses(e_coli_dir, scope_dir_s2lp, objective_dir, list_dir_lev1, list_dir_lev2, list_dir_lev3, list_dir_lev4, "e_coli_core")

           
                       _   ___    _   
  ___   ___   ___   __| | |_  \  | | _ __  
 / __| / _ \ / _ \ / _` |   ) |  | || '_ \ 
 \__ \|  __/|  __/| (_| |  / /_  | || |_) |
 |___/ \___| \___| \__,_| |____| |_|| .__/    
                                    |_|         
      
/home/cghassem/Projets/seed-2-lp/notebook/run/../../scripts/10_1_scope_analyse.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  comparison_scope_df = pd.concat([comparison_scope_df, current_df], ignore_index=True)
           
                       _   ___    _   
  ___   ___   ___   __| | |_  \  | | _ __  
 / __| / _ \ / _ \ / _` |   ) |  | || '_ \ 
 \__ \|  __/|  __/| (_| |  / /_  | || |_) |
 |___/ \___| \___| \__,_| |____| |_|| .__/    
               

## NetSeed

In [59]:
list_dir_lev1=["netseed"]
list_dir_lev2=["netseed"]
list_dir_lev3=["other"]
list_dir_lev4=["accu"]

### Variable to change (if wanted)

In [16]:
scope_dir_netseed=f"{result_dir}/scopes_netseed"

### RUN

In [61]:
run_scope_analyses(e_coli_dir, scope_dir_netseed, objective_dir, list_dir_lev1, list_dir_lev2, list_dir_lev3, list_dir_lev4, "e_coli_core")

           
                       _   ___    _   
  ___   ___   ___   __| | |_  \  | | _ __  
 / __| / _ \ / _ \ / _` |   ) |  | || '_ \ 
 \__ \|  __/|  __/| (_| |  / /_  | || |_) |
 |___/ \___| \___| \__,_| |____| |_|| .__/    
                                    |_|         
      
/home/cghassem/Projets/seed-2-lp/notebook/run/../../scripts/10_1_scope_analyse.py:224: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  comparison_scope_df = pd.concat([comparison_scope_df, current_df], ignore_index=True)


## iCN718

In [43]:
list_dir_lev1=["target"]
list_dir_lev2=["reasoning", "reasoning_filter", "reasoning_guess_check", "reasoning_guess_check_diversity"]
list_dir_lev3=["subset_minimal"]
list_dir_lev4=["no_accu"]

### Variable to change (if wanted)

In [44]:
scope_dir_iCN718=f"{result_dir}/scopes_iCN718"

### RUN

In [45]:
run_scope_analyses(e_coli_dir, scope_dir_s2lp, objective_dir, list_dir_lev1, list_dir_lev2, list_dir_lev3, list_dir_lev4, "e_coli_core")

           
                       _   ___    _   
  ___   ___   ___   __| | |_  \  | | _ __  
 / __| / _ \ / _ \ / _` |   ) |  | || '_ \ 
 \__ \|  __/|  __/| (_| |  / /_  | || |_) |
 |___/ \___| \___| \__,_| |____| |_|| .__/    
                                    |_|         
      
/home/cghassem/Projets/seed-2-lp/notebook/run/../../scripts/10_1_scope_analyse.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  comparison_scope_df = pd.concat([comparison_scope_df, current_df], ignore_index=True)
           
                       _   ___    _   
  ___   ___   ___   __| | |_  \  | | _ __  
 / __| / _ \ / _ \ / _` |   ) |  | || '_ \ 
 \__ \|  __/|  __/| (_| |  / /_  | || |_) |
 |___/ \___| \___| \__,_| |____| |_|| .__/    
               

### RUN

In [66]:
run_scope_analyses(e_coli_dir, scope_dir_iCN718, objective_dir, list_dir_lev1, list_dir_lev2, list_dir_lev3, list_dir_lev4, "iCN718")

           
                       _   ___    _   
  ___   ___   ___   __| | |_  \  | | _ __  
 / __| / _ \ / _ \ / _` |   ) |  | || '_ \ 
 \__ \|  __/|  __/| (_| |  / /_  | || |_) |
 |___/ \___| \___| \__,_| |____| |_|| .__/    
                                    |_|         
      
/home/cghassem/Projets/seed-2-lp/notebook/run/../../scripts/10_1_scope_analyse.py:224: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  comparison_scope_df = pd.concat([comparison_scope_df, current_df], ignore_index=True)
           
                       _   ___    _   
  ___   ___   ___   __| | |_  \  | | _ __  
 / __| / _ \ / _ \ / _` |   ) |  | || '_ \ 
 \__ \|  __/|  __/| (_| |  / /_  | || |_) |
 |___/ \___| \___| \__,_| |____| |_|| .__/    
               

### **List of output files**

In each of scope directories you'll find

- logs directory
- sbml direcotory
- scope direcotory  and in each:
    - a directory "target" (or full wwhen Full Network used but not in our example) or "netseed"  and in each:
        - a directory for each mode (reasoning, filter, Guess-Check or Guess-Check-Div) or "netseed"  and in each:
            - a directory for "submin" (and "minimize" when used) or "other"  and in each:
                - a file *_compare.tsv which contains all scope of the directory

## Concat by scopes directory (al sub repository)

## **Slurm-based cluster**: Reproducing paper data
Slurm-based scripts for cluster are available:
- Launch if needed 
    - all previous scripts to get results for iCN718 and scope, see notebbok: [07_run_seed2lp_iCN718](07_run_seed2lp_iCN718.ipynb)
- Change **_source_** variable by the path of your conda environement with seed2lp installed in files: 
    - [10_2_run_scope_analyse_concat](../../scripts/plafrim_cluster/10_2_run_scope_analyse_concat.sh)
- launch 
    - [10_2_run_scope_analyse_concat](../../scripts/plafrim_cluster/10_2_run_scope_analyse_concat.sh): `sbatch 10_2_run_scope_analyse_concat.`

### RUN

In [62]:
def concat_scopes_by_dir(scope_dir):
    for dir in listdir(scope_dir):
        species=dir
        path_scope=path.join(scope_dir, species, 'scope')
        files_list=list()
        for dirpath, dirnames, filenames in walk(path_scope):
            for filename in [f for f in filenames if f.endswith("_compare.tsv")]:
                files_list.append(path.join(dirpath, filename))
           
    output_file=path.join(scope_dir, f'{species}_scope_compare.tsv')
    first_line=None
    with open(output_file, 'w') as outfile:
        for file in files_list:
            with open(file) as infile:
                if not first_line:
                    first_line = infile.readline()
                    outfile.write(first_line)
                else:
                    next(infile)
                for line in infile:
                    outfile.write(line)

In [63]:
concat_scopes_by_dir(scope_dir_s2lp)

In [64]:
concat_scopes_by_dir(scope_dir_netseed)

In [65]:
concat_scopes_by_dir(scope_dir_iCN718)

### **List of output files**

In each of scope directories you'll find

- logs directory
- sbml direcotory
- scope direcotory and in each:
    -  a file *_scope_compare.tsv which contains all scope of subdirectories

# METABOLITES ANALYSE FOR iCN718

In this section we will count the occurences of metabolites in solutions by Search method Reasoning / Filter / Gues Check and Guess Check Diversity.

## **WARNING**
This section of the notebook will run the metabolites occurences on solution only for iCN718 with 30 solutions.

On the paper, the metabolites occurences was computed for:
- iCN718 Seed2LP results files (up to 2000), Target mode, submin, no accumulation, *Reasoning*, *Hybrid-Filter*, *Hybrid-GC*, *Hybrid-GC<sub>Div<sub>*.


> Note:
>
> - All results file from Seed2LP iCN metabolites occurences are available here: ????? (LINK)

## Requirements
Module *seed2lp* needed

> Advice:
> 
> Use a conda env called s2lp with python 3.10 for plafrim cluster scripts

## **Slurm-based cluster**: Reproducing paper data
Slurm-based scripts for cluster are available:
- Launch if needed 
    - previous scripts to get iCN718 results. Seed notebook : [07_run_seed2lp_iCN718](07_run_seed2lp_iCN718.ipynb)
- Change **_source_** variable by the path of your conda environement with seed2lp installed in files: 
    - [10_3_job_iCN718_metabolite_analysis](../../scripts/slurm_cluster/10_3_job_iCN718_metabolite_analysis.sh)
- launch 
    - [10_3_job_iCN718_metabolite_analysis](../../scripts/slurm_cluster/10_3_job_iCN718_metabolite_analysis.sh): `sbatch 10_3_job_iCN718_metabolite_analysis.sh`


In [1]:
from os import path, makedirs

### Variable to change (if wanted)

In [6]:
analyse_dir = "../../analyses"
data_dir  = f"{analyse_dir}/data/"
result_dir=f"{analyse_dir}/results"

species="iCN718"
sbml_dir = f"{data_dir}/bigg/sbml"
sbml_file = f"{sbml_dir}/{species}.xml"
species_result_dir=f"{result_dir}/{species}_2000/{species}"
output_dir=f"{result_dir}/metabolites_{species}"

In [3]:
if not path.isdir(output_dir):
    makedirs(output_dir)

### RUN

In [8]:
file = "../../scripts/10_3_iCN718_metabolite_analyses.py"

command=f"{file} {species_result_dir} {sbml_file} {output_dir}"
!python {command}


           
                       _   ___    _   
  ___   ___   ___   __| | |_  \  | | _ __  
 / __| / _ \ / _ \ / _` |   ) |  | || '_ \ 
 \__ \|  __/|  __/| (_| |  / /_  | || |_) |
 |___/ \___| \___| \__,_| |____| |_|| .__/    
                                    |_|         
      


### **List of output files**

In the metabolite output directory you will find a file called "metabolites_occurences.tsv"

# GET SOLUTIONS SUPPLEMENTARY DATA

## **Slurm-based cluster**: Reproducing paper data
Slurm-based scripts for cluster are available:
- Launch if needed 
    - all previous scripts to get results, see notebook: 
        - [03_run_seed2lp](03_run_seed2lp.ipynb)
        - [05_run_netseed](05_run_netseed.ipynb)
        - [07_run_seed2lp_iCN718](07_run_seed2lp_iCN718.ipynb)
- Change **_source_** variable by the path of your conda environement with seed2lp installed in files: 
    - [10_4_job_get_supp_data](../../scripts/plafrim_cluster/10_4_job_get_supp_data.sh)
- launch 
    - [10_4_job_get_supp_data](../../scripts/plafrim_cluster/10_4_job_get_supp_data.sh): `sbatch 10_4_job_get_supp_data.sh`


### Variable to change (if wanted)

In [9]:
analyse_dir = "../../analyses"
data_dir  = f"{analyse_dir}/data/"
result_dir=f"{analyse_dir}/results"

sbml_dir=f"{data_dir}/sbml_corrected"
s2lp_result_dir=f"{result_dir}/s2lp"
iCN718_result_dir=f"{result_dir}/iCN718"
onesol_result_dir=f"{result_dir}/one_solution"
out_dir=f"{result_dir}/supp_data"

In [13]:
if not path.isdir(out_dir):
    makedirs(out_dir)

In [11]:
file = "../../scripts/10_4_get_supp_data.py"

## Seed2LP

In [14]:
s2lp_command = f"{file} {s2lp_result_dir} {sbml_dir} {out_dir} 'seed2lp'"
!python {s2lp_command}

           
                       _   ___    _   
  ___   ___   ___   __| | |_  \  | | _ __  
 / __| / _ \ / _ \ / _` |   ) |  | || '_ \ 
 \__ \|  __/|  __/| (_| |  / /_  | || |_) |
 |___/ \___| \___| \__,_| |____| |_|| .__/    
                                    |_|         
      
/home/cghassem/Projets/seed-2-lp/notebook/run/../../scripts/10_4_get_supp_data.py:152: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results=pd.concat([results, current_df], ignore_index=True)


## iCN718

In [15]:
iCN718_command = f"{file} {iCN718_result_dir} {sbml_dir} {out_dir} 'iCN718_2000'"
!python {iCN718_command}

           
                       _   ___    _   
  ___   ___   ___   __| | |_  \  | | _ __  
 / __| / _ \ / _ \ / _` |   ) |  | || '_ \ 
 \__ \|  __/|  __/| (_| |  / /_  | || |_) |
 |___/ \___| \___| \__,_| |____| |_|| .__/    
                                    |_|         
      
/home/cghassem/Projets/seed-2-lp/notebook/run/../../scripts/10_4_get_supp_data.py:152: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results=pd.concat([results, current_df], ignore_index=True)


## One solution

In [16]:
onesol_command = f"{file} {onesol_result_dir} {sbml_dir} {out_dir} 'one_solution'"
!python {onesol_command}

           
                       _   ___    _   
  ___   ___   ___   __| | |_  \  | | _ __  
 / __| / _ \ / _ \ / _` |   ) |  | || '_ \ 
 \__ \|  __/|  __/| (_| |  / /_  | || |_) |
 |___/ \___| \___| \__,_| |____| |_|| .__/    
                                    |_|         
      
/home/cghassem/Projets/seed-2-lp/notebook/run/../../scripts/10_4_get_supp_data.py:152: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results=pd.concat([results, current_df], ignore_index=True)


# GET EXCHANGED METABOLITES WITHOUT PREFIX AND SUFFIX
For iCN 718 metabolites analyses it is needed to retrieve exchanged metabolites without the prefix "M_" and compartment suffix. 

## **Slurm-based cluster**: Reproducing paper data
Slurm-based scripts for cluster are available:
- Launch if needed 
    - get sbml file from BiGG, see notebook:  [01_get_sbml_BiGG](01_get_sbml_BiGG.ipynb)
- Change **_source_** variable by the path of your conda environement with seed2lp installed in files: 
    - [10_5_job_get_iCN718_exchange](../../scripts/slurm_cluster/10_5_job_get_iCN718_exchange.sh)
- launch 
    - [10_5_job_get_iCN718_exchange](../../scripts/plafrim_cluster/10_5_job_get_iCN718_exchange.sh): `sbatch 10_5_job_get_iCN718_exchange.sh`


In [1]:
file = "../../scripts/10_5_get_exchange.py"

In [2]:
species="iCN718"
data_dir="../../data"
sbml_dir = f"{data_dir}/bigg/sbml"
sbml_file = f"{sbml_dir}/{species}.xml"
result_dir="../../results"
output_dir=f"{result_dir}/metabolites_{species}"

In [3]:
command_exch = f"{file} {species} {sbml_file} {output_dir}"

!python {command_exch}

           
                       _   ___    _   
  ___   ___   ___   __| | |_  \  | | _ __  
 / __| / _ \ / _ \ / _` |   ) |  | || '_ \ 
 \__ \|  __/|  __/| (_| |  / /_  | || |_) |
 |___/ \___| \___| \__,_| |____| |_|| .__/    
                                    |_|         
      
